### Mapping predicted counts for 2025 and 2033

In [1]:
# IMPORTS
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from scipy.spatial import cKDTree
from shapely.geometry import Point
import warnings
from prophet import Prophet
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Suppress future warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Suppress deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Get predicted counts data:

In [2]:
csv_names = ['engine_1',
 'engine_2',
 'engine_3',
 'engine_5',
 'engine_7',
 'engine_8',
 'engine_9',
 'engine_10_truck_2',
 'engine_12',
 'engine_13_truck_10',
 'engine_16_truck_6',
 'engine_17_rescue_11',
 'engine_19',
 'truck_3',
 'truck_4',
 'truck_5']

In [3]:
shapefile_path = "/Users/medhinisridharr/Documents/University of Rochester/Courses/Fall 2024/Capstone Project/Model/RFD_Station_Locations/RFD_Station_Locations.shp"
gdf_fire_stations = gpd.read_file(shapefile_path)

In [4]:
predicted_data_dict = {}
for n in csv_names:
    globals()[n] = pd.read_csv(f'/Users/medhinisridharr/Documents/University of Rochester/Courses/Fall 2024/Capstone Project/monthly_incident_counts/Predicted Monthly Incident Counts/Forecasted Data/{n}_historical_forecasted_values.csv')
    predicted_data_dict[n] = globals()[n]

Combine all stations' data into one dataframe for analysis and mapping:

In [5]:
df_monthly_incident_near_rfd_station = pd.DataFrame()
for n in csv_names:
    df = predicted_data_dict[n][['ds','monthly_incidents_per_nearest_station_count']].copy()
    df['nearest_station_name'] = n
    df_monthly_incident_near_rfd_station = pd.concat([df_monthly_incident_near_rfd_station,df],ignore_index=True)

In [6]:
name_dict = {}
for (i,j) in zip(sorted(df_monthly_incident_near_rfd_station['nearest_station_name'].unique()),sorted(gdf_fire_stations['NAME'].unique())):
    name_dict[i] = j

df_monthly_incident_near_rfd_station['nearest_station_name'] = df_monthly_incident_near_rfd_station['nearest_station_name'].map(name_dict)

In [7]:
df_monthly_incident_near_rfd_station

,ds,monthly_incidents_per_nearest_station_count,nearest_station_name
0,2007-01-01,146.000000,Engine 1
1,2007-02-01,131.000000,Engine 1
2,2007-03-01,144.000000,Engine 1
3,2007-04-01,121.000000,Engine 1
4,2007-05-01,122.000000,Engine 1
...,...,...,...
5323,2034-05-01,207.387411,Truck 5
5324,2034-06-01,190.230800,Truck 5
5325,2034-07-01,191.318803,Truck 5
5326,2034-08-01,210.198334,Truck 5


Subset only for period of analysis 2025 and 2033:

In [8]:
df_monthly_incident_near_rfd_station_2025 = df_monthly_incident_near_rfd_station[pd.to_datetime(df_monthly_incident_near_rfd_station['ds']).dt.year == 2025].reset_index(drop=True)
df_monthly_incident_near_rfd_station_2033 = df_monthly_incident_near_rfd_station[pd.to_datetime(df_monthly_incident_near_rfd_station['ds']).dt.year == 2033].reset_index(drop=True)

Display pivot with avg. counts for that year for each station:

In [10]:
pd.pivot_table(df_monthly_incident_near_rfd_station_2025, index='nearest_station_name', values = 'monthly_incidents_per_nearest_station_count', aggfunc='mean').reset_index()

,nearest_station_name,monthly_incidents_per_nearest_station_count
0,Engine 1,142.702663
1,Engine 10 / Truck 2,286.302325
2,Engine 12,51.416867
3,Engine 13 / Truck 10,258.250822
4,Engine 16 / Truck 6,194.129660
5,Engine 17 / Rescue 11,285.393350
6,Engine 19,52.028005
7,Engine 2,254.317445
8,Engine 3,129.986426
9,Engine 5,207.795215


In [11]:
pd.pivot_table(df_monthly_incident_near_rfd_station_2033, index='nearest_station_name', values = 'monthly_incidents_per_nearest_station_count', aggfunc='mean').reset_index()

,nearest_station_name,monthly_incidents_per_nearest_station_count
0,Engine 1,150.274131
1,Engine 10 / Truck 2,372.118487
2,Engine 12,56.676905
3,Engine 13 / Truck 10,257.513972
4,Engine 16 / Truck 6,202.595321
5,Engine 17 / Rescue 11,193.423231
6,Engine 19,60.717241
7,Engine 2,246.038410
8,Engine 3,159.662952
9,Engine 5,241.907289


Create interactive map:

In [9]:
# Ensure the GeoDataFrame is in WGS 84 (EPSG:4326), the coordinate system required for folium
if gdf_fire_stations.crs != "EPSG:4326":
    gdf_fire_stations = gdf_fire_stations.to_crs(epsg=4326)

# Get the centroid of the locations to center the map
map_center = [gdf_fire_stations.geometry.y.mean(), gdf_fire_stations.geometry.x.mean()]

# Create a folium map centered around the fire stations
rfd_stations = folium.Map(location=map_center, zoom_start=12, tiles="Stamen Terrain")

# Add a satellite tile layer
folium.TileLayer('cartodb positron').add_to(rfd_stations)

# Convert 'datetimealarm_month_year' to datetime
df_monthly_incident_near_rfd_station_2025['ds'] = pd.to_datetime(df_monthly_incident_near_rfd_station_2025['ds'], format='%Y-%m-%d').apply(lambda x:x.strftime('%Y-%m'))

# Aggregate monthly incident counts over the years (e.g., take the average)
aggregated_incidents = df_monthly_incident_near_rfd_station_2025.groupby('nearest_station_name')['monthly_incidents_per_nearest_station_count'].mean().reset_index()

# Set min/max values for normalization (to scale the size and opacity of the circles)
min_incident_count = aggregated_incidents['monthly_incidents_per_nearest_station_count'].min()
max_incident_count = aggregated_incidents['monthly_incidents_per_nearest_station_count'].max()

# Normalize function to scale radius and opacity based on incident count
def normalize(value, min_value, max_value):
    return (value - min_value) / (max_value - min_value)

# Plotting each fire station with a density effect using the size and color of dots to represent aggregated incident counts
for _, station in gdf_fire_stations.iterrows():
    station_name = station['NAME']
    
    # Find the corresponding incident count for the station
    incident_info = aggregated_incidents[aggregated_incidents['nearest_station_name'] == station_name]
    
    if not incident_info.empty:
        monthly_incident_count = incident_info.iloc[0]['monthly_incidents_per_nearest_station_count']
        incident_text = f'{monthly_incident_count:.2f} incidents (average per month)'
        
        # Normalize the incident count to set the circle size and opacity
        normalized_size = normalize(monthly_incident_count, min_incident_count, max_incident_count)
        
        # Scale the radius between 5 and 25 based on normalized size
        circle_radius = 5 + (20 * normalized_size)  # Range from 5 to 25
        circle_opacity = 0.3 + (0.7 * normalized_size)  # Opacity from 0.3 to 1.0 based on density
        
        # Set color based on the normalized density
        if normalized_size < 0.33:
            circle_color = 'lightblue'  # Low density
        elif normalized_size < 0.66:
            circle_color = 'lightgreen'  # Medium density
        else:
            circle_color = 'orange'  # High density
    else:
        incident_text = 'No data available'
        circle_radius = 5
        circle_opacity = 0.3
        circle_color = 'gray'  # No data color
    
    # Add a red fire station marker
    folium.Marker(
        location=[station.geometry.y, station.geometry.x],
        icon=folium.Icon(icon='fire', prefix='fa', color='red'),
        popup=station_name  # Popup showing the fire station's name
    ).add_to(rfd_stations)

    # Add a colored dot at the station location to represent the density of incidents
    folium.CircleMarker(
        location=[station.geometry.y, station.geometry.x],
        radius=circle_radius,  # Size of the circle marker based on the incident count
        color=circle_color,  # Border color based on density
        fill=True,
        fill_color=circle_color,  # Fill color for the circle marker
        fill_opacity=circle_opacity,  # Vary opacity based on incident count
        popup=incident_text  # Popup showing the incident count
    ).add_to(rfd_stations)

# Add a title to the map
title_html = '''
             <h3 align="center" style="font-size:20px"><b>Rochester Fire Department Stations and Avg. Forecasted Monthly Incident Density - 2025</b></h3>
             '''
rfd_stations.get_root().html.add_child(folium.Element(title_html))

# Add a custom legend
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 200px; height: 150px; 
                 background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
                 ">&nbsp;<b>Incident Density Levels</b><br>
                   &nbsp;<i class="fa fa-circle" style="color:lightblue"></i>&nbsp; Low Density<br>
                   &nbsp;<i class="fa fa-circle" style="color:lightgreen"></i>&nbsp; Medium Density<br>
                   &nbsp;<i class="fa fa-circle" style="color:orange"></i>&nbsp; High Density
     </div>
     '''
rfd_stations.get_root().html.add_child(folium.Element(legend_html))

# Display the map
rfd_stations

In [10]:
# Ensure the GeoDataFrame is in WGS 84 (EPSG:4326), the coordinate system required for folium
if gdf_fire_stations.crs != "EPSG:4326":
    gdf_fire_stations = gdf_fire_stations.to_crs(epsg=4326)

# Get the centroid of the locations to center the map
map_center = [gdf_fire_stations.geometry.y.mean(), gdf_fire_stations.geometry.x.mean()]

# Create a folium map centered around the fire stations
rfd_stations = folium.Map(location=map_center, zoom_start=12, tiles="Stamen Terrain")

# Add a satellite tile layer
folium.TileLayer('cartodb positron').add_to(rfd_stations)

# Convert 'datetimealarm_month_year' to datetime
df_monthly_incident_near_rfd_station_2033['ds'] = pd.to_datetime(df_monthly_incident_near_rfd_station_2033['ds'], format='%Y-%m-%d').apply(lambda x:x.strftime('%Y-%m'))

# Aggregate monthly incident counts over the years (e.g., take the average)
aggregated_incidents = df_monthly_incident_near_rfd_station_2033.groupby('nearest_station_name')['monthly_incidents_per_nearest_station_count'].mean().reset_index()

# Set min/max values for normalization (to scale the size and opacity of the circles)
min_incident_count = aggregated_incidents['monthly_incidents_per_nearest_station_count'].min()
max_incident_count = aggregated_incidents['monthly_incidents_per_nearest_station_count'].max()

# Normalize function to scale radius and opacity based on incident count
def normalize(value, min_value, max_value):
    return (value - min_value) / (max_value - min_value)

# Plotting each fire station with a density effect using the size and color of dots to represent aggregated incident counts
for _, station in gdf_fire_stations.iterrows():
    station_name = station['NAME']
    
    # Find the corresponding incident count for the station
    incident_info = aggregated_incidents[aggregated_incidents['nearest_station_name'] == station_name]
    
    if not incident_info.empty:
        monthly_incident_count = incident_info.iloc[0]['monthly_incidents_per_nearest_station_count']
        incident_text = f'{monthly_incident_count:.2f} incidents (average per month)'
        
        # Normalize the incident count to set the circle size and opacity
        normalized_size = normalize(monthly_incident_count, min_incident_count, max_incident_count)
        
        # Scale the radius between 5 and 25 based on normalized size
        circle_radius = 5 + (20 * normalized_size)  # Range from 5 to 25
        circle_opacity = 0.3 + (0.7 * normalized_size)  # Opacity from 0.3 to 1.0 based on density
        
        # Set color based on the normalized density
        if normalized_size < 0.33:
            circle_color = 'lightblue'  # Low density
        elif normalized_size < 0.66:
            circle_color = 'lightgreen'  # Medium density
        else:
            circle_color = 'orange'  # High density
    else:
        incident_text = 'No data available'
        circle_radius = 5
        circle_opacity = 0.3
        circle_color = 'gray'  # No data color
    
    # Add a red fire station marker
    folium.Marker(
        location=[station.geometry.y, station.geometry.x],
        icon=folium.Icon(icon='fire', prefix='fa', color='red'),
        popup=station_name  # Popup showing the fire station's name
    ).add_to(rfd_stations)

    # Add a colored dot at the station location to represent the density of incidents
    folium.CircleMarker(
        location=[station.geometry.y, station.geometry.x],
        radius=circle_radius,  # Size of the circle marker based on the incident count
        color=circle_color,  # Border color based on density
        fill=True,
        fill_color=circle_color,  # Fill color for the circle marker
        fill_opacity=circle_opacity,  # Vary opacity based on incident count
        popup=incident_text  # Popup showing the incident count
    ).add_to(rfd_stations)

# Add a title to the map
title_html = '''
             <h3 align="center" style="font-size:20px"><b>Rochester Fire Department Stations and Avg. Forecasted Monthly Incident Density - 2033</b></h3>
             '''
rfd_stations.get_root().html.add_child(folium.Element(title_html))

# Add a custom legend
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 200px; height: 150px; 
                 background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
                 ">&nbsp;<b>Incident Density Levels</b><br>
                   &nbsp;<i class="fa fa-circle" style="color:lightblue"></i>&nbsp; Low Density<br>
                   &nbsp;<i class="fa fa-circle" style="color:lightgreen"></i>&nbsp; Medium Density<br>
                   &nbsp;<i class="fa fa-circle" style="color:orange"></i>&nbsp; High Density
     </div>
     '''
rfd_stations.get_root().html.add_child(folium.Element(legend_html))

# Display the map
rfd_stations